In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

from __future__ import print_function, division
import PIL
import numpy as np
import tensorflow as tf
tf.config.run_functions_eagerly(True)
# load and show an image with Pillow
from PIL import Image
from os import listdir


import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy
# load all images in a directory
# load all images in a directory

BASE_DIR = '/content/gdrive/Shareddrives/520 - Machine Learning Project/Dataset/'
SMALL_NORMAL_DIR = os.path.join(BASE_DIR + 'Normal2/')
SMALL_PNEUMONIA_DIR = os.path.join(BASE_DIR + 'Viral_Pneumonia2/')
SMALL_COVID_DIR = os.path.join(BASE_DIR + 'COVID2/')
dir_normal = os.path.join(BASE_DIR + 'Normal2/')
dir_covid = os.path.join(BASE_DIR + 'Viral_Pneumonia2/')
dir_pneumonia = os.path.join(BASE_DIR + 'COVID2/')
loaded_images = []
labels = []

filelist = dir_normal, dir_covid, dir_pneumonia
#loaded_images = np.array([np.array((Image.open(fname)).convert("RGB") for fname in filelist])

for filename in listdir(dir_normal):
	# load image
	img_data = Image.open(dir_normal + filename)
	img_data = img_data.convert("RGB")
	# store loaded image
	loaded_images.append(img_data)
	labels.append(0)
for filename in listdir(dir_covid):
  # load image
	img_data = Image.open(dir_covid + filename)
	img_data = img_data.convert("RGB")
	# store loaded image
	loaded_images.append(img_data)
	labels.append(1)
for filename in listdir(dir_pneumonia):
  # load image
	img_data = Image.open(dir_pneumonia + filename)
	img_data = img_data.convert("RGB")
	# store loaded image
	loaded_images.append(img_data)
	labels.append(2)

Mounted at /content/gdrive


In [ ]:
images = loaded_images
prepared_images = []
preprocess = transforms.Compose([
    transforms.Resize(299),
    transforms.CenterCrop(299),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

input_tensors = []
for i in range(len(loaded_images)):
  input_tensor = preprocess(loaded_images[i])
  input_tensors.append(input_tensor)



prep_images = torch.stack((input_tensors))

In [ ]:
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.optimizers import Adam
pre_model = InceptionV3(input_shape = (299,299,3), include_top = False, weights = 'imagenet')
for layer in pre_model.layers:
  layer.trainable = False

x = tf.keras.layers.Flatten()(pre_model.output)
x = tf.keras.layers.Dense(1024, activation = 'relu')(x)
x = tf.keras.layers.Dropout(0.1)(x)
x = tf.keras.layers.Dense(1,activation='sigmoid')(x)

model = tf.keras.Model(pre_model.input,x)
model.compile(optimizer=Adam(lr=.0001),
              loss = 'binary_crossentropy',
              metrics = ['acc'])

In [ ]:
y_covid = [i for i in labels if i == 0 or i == 1]
x_covid = prep_images[0:len(y_covid)]
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x_covid, y_covid, test_size=0.33, random_state=42)

In [ ]:
X_train.shape

torch.Size([536, 3, 299, 299])

In [ ]:
history = model.fit(np.reshape(X_train.numpy(),(536,299,299,3)),np.array(y_train),steps_per_epoch=30,epochs=3)

/opt/anaconda3/envs/newpy/lib/python3.6/site-packages/tensorflow/python/data/ops/dataset_ops.py:3504: UserWarning: Even though the tf.config.experimental_run_functions_eagerly option is set, this option does not apply to tf.data functions. tf.data functions are still traced and executed as graphs.
  "Even though the tf.config.experimental_run_functions_eagerly "


Epoch 1/3
30/30 [==============================] - 230s 8s/step - loss: 3.6716 - acc: 0.7580
Epoch 2/3
30/30 [==============================] - 226s 8s/step - loss: 0.3048 - acc: 0.9219
Epoch 3/3
30/30 [==============================] - 224s 7s/step - loss: 0.1697 - acc: 0.9485


In [ ]:
#X_test.shape
y_prob = model.predict(np.reshape(X_test.numpy(),(264,299,299,3)))

In [ ]:
import sklearn.metrics as metrics
from sklearn.metrics import f1_score

auc = metrics.roc_auc_score(y_test, y_prob)
print('ROC AUC: %f' % auc)

ROC AUC: 0.985149
